## btc_price_with_indicators

In [4]:
import pandas as pd
import numpy as np

# 데이터 불러오기, 시계열데이터로 변환
df=pd.read_csv('../data/BIT_2023_1차.csv')
df.set_index('time', inplace=True)
df.index=pd.to_datetime(df.index)

In [5]:
# MACD 함수
def MACD(df, short_window, long_window, signal_window):
    
    df[f'Short_MA_{short_window}'] = df['Close'].ewm(span=short_window, adjust=False).mean()
    df[f'Long_MA_{long_window}'] = df['Close'].ewm(span=long_window, adjust=False).mean()
    df[f'MACD_{short_window}{long_window}'] = df[f'Short_MA_{short_window}'] - df[f'Long_MA_{long_window}']
    df[f'Signal_Line_{signal_window}'] = df[f'MACD_{short_window}{long_window}'].ewm(span=signal_window, adjust=False).mean()
    df[f'MACD_Oscillator{short_window}{long_window}'] = df[f'MACD_{short_window}{long_window}'] - df[f'Signal_Line_{signal_window}']
    
# Stochastic Oscillator 함수
def cal_stoc_os(df, period=14, m=3): # period = 14 or 20
    
    # 최근 N일간의 최고가(H)와 최저가(L)
    df['highest_high'] = df['High'].rolling(window=period).max()
    df['lowest_low'] = df['Low'].rolling(window=period).min()
    
    # %K 계산
    df['%K'] = (df['Close'] - df['lowest_low']) / (df['highest_high'] - df['lowest_low']) * 100
    
    # %D 계산
    df['%D'] = df['%K'].rolling(m).mean()
    
# ATR 함수
def calculate_atr(df, period=14):
    # True Range 계산
    df['high-low'] = df['High'] - df['Low']
    df['high-close'] = abs(df['High'] - df['Close'].shift())
    df['low-close'] = abs(df['Low'] - df['Close'].shift())
    
    df['true_range'] = df[['high-low', 'high-close', 'low-close']].max(axis=1)
    
    # ATR 계산
    df['atr'] = df['true_range'].rolling(window=period, min_periods=1).mean()
    
    # 임시 열 삭제
    df.drop(['high-low', 'high-close', 'low-close', 'true_range'], axis=1, inplace=True)

def bollinger_bands(df, window=20, num_std_dev=2):
    
    # 이동평균 계산
    df['rolling_mean'] = df['price'].rolling(window=window).mean()

    # 표준편차 계산
    df['rolling_std'] = df['price'].rolling(window=window).std()

    # 볼린저 밴드 상단 및 하단 계산
    df['upper_band'] = df['rolling_mean'] + (num_std_dev * df['rolling_std'])
    df['lower_band'] = df['rolling_mean'] - (num_std_dev * df['rolling_std'])

    # 필요없는 컬럼 삭제
    df.drop(['rolling_mean', 'rolling_std'], axis=1, inplace=True)

In [6]:
MACD(df, 6, 13, 4) #단기추세 감지 but 노이즈에 민감할 가능성 있음
MACD(df, 12, 26, 9) #일반적인 window 계수 but 변동성이 큰 시장에선 느리게 반응할 수 있음
calculate_atr(df)
cal_stoc_os(df)
bollinger_bands(df)

In [7]:
df.columns

Index(['Open', 'High', 'Low', 'Close', 'Volume', 'returns', 'volatility',
       'price', 'quote_qty', 'is_buyer_maker', 'quote_qty_cal', 'Short_MA_6',
       'Long_MA_13', 'MACD_613', 'Signal_Line_4', 'MACD_Oscillator613',
       'Short_MA_12', 'Long_MA_26', 'MACD_1226', 'Signal_Line_9',
       'MACD_Oscillator1226', 'atr', 'highest_high', 'lowest_low', '%K', '%D',
       'upper_band', 'lower_band'],
      dtype='object')

In [11]:
df.to_csv('../data/BIT_2023_2차.csv', encoding = 'utf-8-sig')